In [1]:
import pandas as pd
import numpy

In [2]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Checking for missing values in the dataset
missing_values = data.isnull().sum()

# Impute missing values for 'bmi' using the median
imputer = SimpleImputer(strategy='median')
data['bmi'] = imputer.fit_transform(data[['bmi']])

# Encode categorical columns using LabelEncoder
label_encoder = LabelEncoder()
categorical_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])

# Display the cleaned dataset summary
data.info(), data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   int64  
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   int64  
 6   work_type          5110 non-null   int64  
 7   Residence_type     5110 non-null   int64  
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                5110 non-null   float64
 10  smoking_status     5110 non-null   int64  
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(9)
memory usage: 479.2 KB


(None,
       id  gender   age  hypertension  heart_disease  ever_married  work_type  \
 0   9046       1  67.0             0              1             1          2   
 1  51676       0  61.0             0              0             1          3   
 2  31112       1  80.0             0              1             1          2   
 3  60182       0  49.0             0              0             1          2   
 4   1665       0  79.0             1              0             1          3   
 
    Residence_type  avg_glucose_level   bmi  smoking_status  stroke  
 0               1             228.69  36.6               1       1  
 1               0             202.21  28.1               2       1  
 2               0             105.92  32.5               2       1  
 3               1             171.23  34.4               3       1  
 4               0             174.12  24.0               2       1  )

In [4]:
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,28.1,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.0,2,1


In [5]:
X = data.drop('id', axis = 1)

In [6]:
X[:1]

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.6,1,1


In [7]:
y = data['stroke']

In [8]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: stroke, dtype: int64

In [9]:
print(data['stroke'].value_counts())

stroke
0    4861
1     249
Name: count, dtype: int64


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Display the shape of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4088, 11), (1022, 11), (4088,), (1022,))

# Approach 1- Using weighted classes

In [11]:
import torch
from torch import nn

In [12]:
y_train.dtype

dtype('int64')

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
device 

'cuda'

In [15]:
y_test.dtype

dtype('int64')

In [16]:
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)

In [17]:
X_test[:1]

tensor([[ 1.1876,  0.8696, -0.3280, -0.2391,  0.7170, -0.1611, -1.0123, -0.6207,
          0.7616,  0.5892, -0.2262]], dtype=torch.float64)

In [18]:
y_train.dtype

torch.int64

In [19]:
X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

In [20]:
X_train.shape

torch.Size([4088, 11])

## Preparing baseline model

In [21]:
class BaseLineModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.LinearBlock1 = nn.Sequential(
            nn.Linear(in_features = 11,
                      out_features = 32),
            nn.Linear(in_features = 32,
                      out_features = 32),
            nn.ReLU()
        )
        self.LinearBlock2 = nn.Sequential(
            nn.Linear(in_features = 32,
                      out_features = 16),
            nn.Linear(in_features = 16,
                      out_features = 1),
            nn.ReLU()
        )

    def forward(self, x):
        """
        x = self.LinearBlock1(x)
        print(x)
        x = self.LinearBlock2(x)
        print(x)
        """
        return self.LinearBlock2(self.LinearBlock1(x))


model0 = BaseLineModel().to(device)

In [22]:
model0

BaseLineModel(
  (LinearBlock1): Sequential(
    (0): Linear(in_features=11, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): ReLU()
  )
  (LinearBlock2): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=1, bias=True)
    (2): ReLU()
  )
)

In [23]:
from torchinfo import summary

In [24]:
summary(model0)

Layer (type:depth-idx)                   Param #
BaseLineModel                            --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       384
│    └─Linear: 2-2                       1,056
│    └─ReLU: 2-3                         --
├─Sequential: 1-2                        --
│    └─Linear: 2-4                       528
│    └─Linear: 2-5                       17
│    └─ReLU: 2-6                         --
Total params: 1,985
Trainable params: 1,985
Non-trainable params: 0

In [25]:
dummy_x = X_train[:1].to(device).to(torch.float32)

In [26]:
dummy_x

tensor([[-0.8403,  0.2057, -0.3280, -0.2391,  0.7170, -0.1611,  0.9878, -0.8200,
          0.5426,  0.5892, -0.2262]], device='cuda:0')

In [27]:
X_train.dtype

torch.float64

In [28]:
X_train = X_train.to(torch.float32)
X_test = X_test.to(torch.float32)
y_test = y_test.to(torch.float32)
y_train = y_train.to(torch.float32)

In [29]:
print(X_train.dtype)
print(X_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

torch.float32
torch.float32
torch.float32
torch.float32


In [30]:
import torch
dummy_y = torch.softmax(model0(dummy_x), 0)
dummy_y.dtype

torch.float32

In [31]:
negs = (y_train == 0).sum()
pos = (y_train == 1).sum()
weight = torch.tensor(negs / pos).to(device)
weight = torch.tensor([15]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight = weight)
optimizer = torch.optim.SGD(lr = 0.0001,
                            momentum = 0.09,
                            params = model0.parameters())


def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


/tmp/ipykernel_3926/3395050998.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weight = torch.tensor(negs / pos).to(device)


In [33]:
EPOCHS = 10000
test_loss_val = []
train_loss_val = []
test_acc_val = []
train_acc_val = []
epoch_count = []
model0.to(device)
for epoch in range(EPOCHS):
    #Training model
    model0.train()
    X_train, y_train = X_train.to(device), y_train.to(device)
    y_logits = model0(X_train).squeeze(1)
    y_pred = torch.sigmoid(y_logits).round()
    #print(y_logits.shape)
    
    train_loss = loss_fn(y_logits, y_train)
    
    train_acc = accuracy_fn(y_train, y_logits)

    optimizer.zero_grad()

    train_loss.backward()

    optimizer.step()

    #Evaluating model
    model0.eval()
    X_test, y_test = X_test.to(device), y_test.to(device)
    with torch.inference_mode():
        test_logits = model0(X_test).squeeze(1)
        test_preds = torch.sigmoid(test_logits).round()

        test_loss = loss_fn(test_logits, y_test)

        test_acc = accuracy_fn(y_test, test_logits)

        if epoch % 1000 == 0:
            epoch_count.append(epoch)
            test_loss_val.append(test_loss)
            test_acc_val.append(test_acc)
            train_loss_val.append(train_loss)
            train_acc_val.append(train_acc)
            print(f"Epoch: {epoch} | Train acc: {train_acc: .2f}% | Train loss: {train_loss: .4f} | Test acc: {test_acc: .2f}% | Test loss: {test_loss: .4f}")

            
        
    

    


    

Epoch: 0 | Train acc:  1.57% | Train loss:  1.1384 | Test acc:  1.27% | Test loss:  1.1398
Epoch: 1000 | Train acc:  9.54% | Train loss:  1.1092 | Test acc:  9.20% | Test loss:  1.1098
Epoch: 2000 | Train acc:  17.86% | Train loss:  1.0790 | Test acc:  17.03% | Test loss:  1.0792
Epoch: 3000 | Train acc:  24.34% | Train loss:  1.0461 | Test acc:  24.66% | Test loss:  1.0460
Epoch: 4000 | Train acc:  32.02% | Train loss:  1.0089 | Test acc:  33.86% | Test loss:  1.0087
Epoch: 5000 | Train acc:  38.75% | Train loss:  0.9680 | Test acc:  40.80% | Test loss:  0.9677
Epoch: 6000 | Train acc:  45.03% | Train loss:  0.9247 | Test acc:  45.30% | Test loss:  0.9242
Epoch: 7000 | Train acc:  50.98% | Train loss:  0.8818 | Test acc:  52.25% | Test loss:  0.8810
Epoch: 8000 | Train acc:  56.04% | Train loss:  0.8418 | Test acc:  56.65% | Test loss:  0.8409
Epoch: 9000 | Train acc:  61.33% | Train loss:  0.8070 | Test acc:  61.35% | Test loss:  0.8061


## Evaluating model

In [34]:
import numpy as np
eval_preds = []
y_true = []

X_test, y_test = X_test.to('cpu'), y_test.to('cpu')
model0.to('cpu')
model0.eval()
with torch.inference_mode():
    eval_logits = model0(X_test).squeeze(1)
    eval_pred = torch.sigmoid(eval_logits).round()

    eval_preds.append(eval_pred)
    y_true.append(y_test)
    y_pred_np = torch.cat(eval_preds).numpy()
    y_true_np = torch.cat(y_true).numpy()


In [35]:
y_pred_np

array([0., 0., 1., ..., 1., 0., 0.], shape=(1022,), dtype=float32)

In [36]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(y_true_np, y_pred_np)

# Optional: Define class names
class_names = ['0', '1']

In [37]:
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Approach 2- Using SMOTE and Undersampling(SMOTEEN)

In [38]:
from imblearn.combine import SMOTEENN
sme = SMOTEENN()
X_re, y_re = sme.fit_resample(X, y)

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_re, y_re, test_size=0.2, random_state=42, stratify=y_re)

# Scale the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Display the shape of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6734, 11), (1684, 11), (6734,), (1684,))

In [40]:
X_train.shape, y_train.shape

((6734, 11), (6734,))

In [41]:
unique_values, counts = np.unique(y_train, return_counts=True)

print(f"Unique values for y_train")
print(f"Unique values:  {unique_values}")
print(f"Counts:         {counts}")


Unique values for y_train
Unique values:  [0 1]
Counts:         [3010 3724]


In [43]:
y_train.dtype

dtype('int64')

In [44]:
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)

In [45]:
X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

In [46]:
X_train = X_train.to(torch.float32)
X_test = X_test.to(torch.float32)
y_test = y_test.to(torch.float32)
y_train = y_train.to(torch.float32)

In [47]:
print(X_train.dtype)
print(X_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

torch.float32
torch.float32
torch.float32
torch.float32


## Training without weighted classes

In [48]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(lr = 0.01,
                            momentum = 0.09,
                            params = model0.parameters())


def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


In [49]:
EPOCHS = 500
test_loss_val = []
train_loss_val = []
test_acc_val = []
train_acc_val = []
epoch_count = []
model0.to(device)
for epoch in range(EPOCHS):
    #Training model
    model0.train()
    X_train, y_train = X_train.to(device), y_train.to(device)
    y_logits = model0(X_train).squeeze(1)
    y_pred = torch.sigmoid(y_logits).round()
    #print(y_logits.shape)
    
    train_loss = loss_fn(y_logits, y_train)
    
    train_acc = accuracy_fn(y_train, y_logits)

    optimizer.zero_grad()

    train_loss.backward()

    optimizer.step()

    #Evaluating model
    model0.eval()
    X_test, y_test = X_test.to(device), y_test.to(device)
    with torch.inference_mode():
        test_logits = model0(X_test).squeeze(1)
        test_preds = torch.sigmoid(test_logits).round()

        test_loss = loss_fn(test_logits, y_test)

        test_acc = accuracy_fn(y_test, test_logits)

        if epoch % 50 == 0:
            epoch_count.append(epoch)
            test_loss_val.append(test_loss)
            test_acc_val.append(test_acc)
            train_loss_val.append(train_loss)
            train_acc_val.append(train_acc)
            print(f"Epoch: {epoch} | Train acc: {train_acc: .2f}% | Train loss: {train_loss: .4f} | Test acc: {test_acc: .2f}% | Test loss: {test_loss: .4f}")

Epoch: 0 | Train acc:  44.46% | Train loss:  0.5877 | Test acc:  44.36% | Test loss:  0.5867
Epoch: 50 | Train acc:  32.88% | Train loss:  0.4882 | Test acc:  32.90% | Test loss:  0.4876
Epoch: 100 | Train acc:  31.23% | Train loss:  0.4262 | Test acc:  32.13% | Test loss:  0.4260
Epoch: 150 | Train acc:  34.81% | Train loss:  0.3846 | Test acc:  35.69% | Test loss:  0.3847
Epoch: 200 | Train acc:  37.56% | Train loss:  0.3592 | Test acc:  38.36% | Test loss:  0.3597
Epoch: 250 | Train acc:  39.65% | Train loss:  0.3442 | Test acc:  40.14% | Test loss:  0.3449
Epoch: 300 | Train acc:  40.85% | Train loss:  0.3349 | Test acc:  41.03% | Test loss:  0.3357
Epoch: 350 | Train acc:  41.89% | Train loss:  0.3289 | Test acc:  41.75% | Test loss:  0.3297
Epoch: 400 | Train acc:  42.43% | Train loss:  0.3248 | Test acc:  42.10% | Test loss:  0.3257
Epoch: 450 | Train acc:  42.81% | Train loss:  0.3220 | Test acc:  42.52% | Test loss:  0.3228


In [50]:
import numpy as np
eval_preds = []
y_true = []

X_test, y_test = X_test.to('cpu'), y_test.to('cpu')
model0.to('cpu')
model0.eval()
with torch.inference_mode():
    eval_logits = model0(X_test).squeeze(1)
    eval_pred = torch.sigmoid(eval_logits).round()

    eval_preds.append(eval_pred)
    y_true.append(y_test)
    y_pred_np = torch.cat(eval_preds).numpy()
    y_true_np = torch.cat(y_true).numpy()


In [51]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(y_true_np, y_pred_np)

# Optional: Define class names
class_names = ['0', '1']

In [52]:
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()